In [3]:
#Import Data Analysis Libraries
import os
import pandas as pd
#os.chdir("")
import numpy as np

In [4]:
#Import Visualization Libraries

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE  # Helps visualize higher dimensions data
import missingno as msno # Missing Data visualization
import seaborn as sns
sns.set(style="white")
%matplotlib inline

In [20]:
import sklearn

In [5]:
#Import Sklearn (Machine learning framework in Python)
from sklearn.preprocessing import StandardScaler, normalize  # Helps us in Standardizing the data
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score, average_precision_score # Evaluation
                                                                                                             # metrics
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.cluster import KMeans # Clustering
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE # Oversampling

In [150]:
train_data=pd.read_csv("C:\\Users\\jon.hai\\Desktop\\train_data.csv",na_values=["?",","])
test_data=pd.read_csv("C:\\Users\\jon.hai\\Desktop\\test_data.csv",na_values=["?",","])
print(train_data.shape)
print(type(train_data))
print(test_data.shape)
print(type(test_data))

(31587, 18)
<class 'pandas.core.frame.DataFrame'>
(976, 17)
<class 'pandas.core.frame.DataFrame'>


In [151]:
train_data.head()

,index,age,working_sector,financial_weight,qualification,years_of_education,tax_paid,loan_taken,marital_status,occupation,relationship,ethnicity,gender,gain,loss,working_hours,country,target
0,1,39.0,state,77516.0,Bachelors,13,NaN,0,Never-married,clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,0
1,2,50.0,self_employed,83311.0,Bachelors,13,NaN,0,Married-civilian,managerial,Husband,White,Male,0.0,0.0,13.0,United-States,0
2,3,38.0,private,215646.0,HS-grad,9,NaN,0,Divorced,cleaner,Not-in-family,White,Male,0.0,0.0,40.0,United-States,0
3,4,53.0,private,234721.0,11th,7,NaN,0,Married-civilian,cleaner,Husband,Black,Male,0.0,0.0,40.0,United-States,0
4,5,28.0,private,338409.0,Bachelors,13,NaN,0,Married-civilian,specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,0


In [152]:
train_data["target"].value_counts()


0    23968
1     7619
Name: target, dtype: int64

In [153]:
train_data.dtypes

index                   int64
age                   float64
working_sector         object
financial_weight      float64
qualification          object
years_of_education      int64
tax_paid              float64
loan_taken              int64
marital_status         object
occupation             object
relationship           object
ethnicity              object
gender                 object
gain                  float64
loss                  float64
working_hours         float64
country                object
target                  int64
dtype: object

In [154]:
train_data=train_data.drop(['index','tax_paid','qualification'],axis=1)
#test_data.drop('index','tax_paid','qualification', axis=1)
train_data['loan_taken']=train_data['loan_taken'].astype('category')
train_data['target']=train_data['target'].astype('category')

In [155]:

train_data['loan_taken']=train_data['loan_taken'].astype('category')
train_data['target']=train_data['target'].astype('category')

In [156]:

cat_train_data=train_data.select_dtypes(include=['category','object'])


In [157]:

for i in cat_train_data:
    train_data[i] = train_data[i].astype('category')


In [158]:
cat_train_data.dtypes

working_sector      object
loan_taken        category
marital_status      object
occupation          object
relationship        object
ethnicity           object
gender              object
country             object
target            category
dtype: object

In [159]:
for i in cat_train_data:
    cat_train_data[i]=cat_train_data[i].fillna(value=cat_train_data[i].mode()[0])

C:\Users\jon.hai\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [160]:
cat_train_data.isnull().sum()

working_sector    0
loan_taken        0
marital_status    0
occupation        0
relationship      0
ethnicity         0
gender            0
country           0
target            0
dtype: int64

In [161]:
from sklearn.preprocessing import LabelEncoder
string_cat = 'marital_status occupation relationship gender working_sector ethnicity country'
cat_features = string_cat.split(' ')
for ft in cat_features:
    le = LabelEncoder()
    cat_train_data[ft] = le.fit_transform(cat_train_data[ft])
    #data_test[ft]=le.fit_transform(data_test[ft])
    

C:\Users\jon.hai\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [162]:
cat_train_data

,working_sector,loan_taken,marital_status,occupation,relationship,ethnicity,gender,country,target
0,5,0,4,1,1,4,1,38,0
1,4,0,1,7,0,4,1,38,0
2,3,0,0,0,1,4,1,38,0
3,3,0,1,0,0,2,1,38,0
4,3,0,1,11,5,2,0,4,0
5,3,0,1,7,5,4,0,38,0
6,3,0,3,8,1,2,0,22,0
7,4,1,1,7,0,4,1,38,1
8,3,0,4,11,1,4,0,38,1
9,3,0,1,7,0,4,1,38,1


In [163]:

num_train_data=train_data.select_dtypes(include=['int','float'])          
num_train_data


,age,financial_weight,gain,loss,working_hours
0,39.0,77516.0,2174.0,0.0,40.0
1,50.0,83311.0,0.0,0.0,13.0
2,38.0,215646.0,0.0,0.0,40.0
3,53.0,234721.0,0.0,0.0,40.0
4,28.0,338409.0,0.0,0.0,40.0
5,37.0,284582.0,0.0,0.0,40.0
6,49.0,160187.0,0.0,0.0,16.0
7,52.0,209642.0,0.0,0.0,45.0
8,31.0,45781.0,14084.0,0.0,50.0
9,42.0,159449.0,5178.0,0.0,40.0


In [164]:
num_train_data=num_train_data.fillna(num_train_data.mean())
num_train_data.isnull().values.sum()


0

In [165]:

train_data= pd.concat([num_train_data, cat_train_data], axis=1, join_axes=[num_train_data.index])
train_data.shape

(31587, 14)

In [166]:
train_data.isnull().values.sum()

0

In [167]:
train_data.dtypes

age                  float64
financial_weight     float64
gain                 float64
loss                 float64
working_hours        float64
working_sector         int32
loan_taken          category
marital_status         int32
occupation             int32
relationship           int32
ethnicity              int32
gender                 int32
country                int32
target              category
dtype: object

In [168]:
y=train_data["target"]
X=train_data.drop('target', axis=1)


#from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)  

In [169]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25269, 13)
(6318, 13)
(25269,)
(6318,)


In [170]:
X_train.isnull().sum().sum()

0

In [ ]:
 from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [ ]:
models = []
names = ['LR','Random Forest','GaussianNB','DecisionTreeClassifier','SVM',]

models.append((LogisticRegression()))
models.append((RandomForestClassifier(n_estimators=100)))
models.append((GaussianNB()))
models.append((DecisionTreeClassifier()))
models.append((SVC()))

In [179]:
print (models)

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), GaussianNB(priors=None, var_smoothing=1e-09), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_w

In [182]:
from sklearn import model_selection
from sklearn.metrics import accuracy_score

In [183]:
kfold = model_selection.KFold(n_splits=5,random_state=7)

for i in range(0,len(models)):    
    cv_result = model_selection.cross_val_score(models[i],X_train,y_train,cv=kfold,scoring='accuracy')
    score=models[i].fit(X_train,y_train)
    prediction = models[i].predict(X_test)
    acc_score = accuracy_score(y_test,prediction)     
    print ('-'*40)
    print ('{0}: {1}'.format(names[i],acc_score))

C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

----------------------------------------
LR: 0.8026274137385249
----------------------------------------
Random Forest: 0.8529597974042419
----------------------------------------
GaussianNB: 0.8027856916745806
----------------------------------------
DecisionTreeClassifier: 0.8137068692624249


C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\jon.hai\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: Fut

----------------------------------------
SVM: 0.7621082621082621


In [ ]:
df = pd.DataFrame(prediction_test)
out_csv = 'C:\Users\jon.hai\Desktop\New folder/python/new-predictions.csv'
df.to_csv(out_csv, index=False, header=False)
print("Predictions saved to disk: {0}".format(out_csv))